In [1]:
! pip install opencv-python==4.5.5.64

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 60.5 MB 100 kB/s 
  Attempting uninstall: opencv-python
    Found existing installation: opencv-python 4.6.0.66
    Uninstalling opencv-python-4.6.0.66:
      Successfully uninstalled opencv-python-4.6.0.66


In [2]:
import sys
from google.colab import drive

drive.mount('/content/drive')
sys.path.append(r"/content/drive/MyDrive/")

Mounted at /content/drive


In [3]:
from loader_util.preprocessing import ImageToArrayPreprocessor
from loader_util.preprocessing import SimplePreProcessor
from loader_util.preprocessing import MeanPreprocessor
from loader_util.callbacks import EpochCheckpoint
from loader_util.callbacks import TrainingMonitor
from loader_util.io import HDF5DatasetGenerator
from loader_util.nn.conv import DeeperGoogleNet

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.models import load_model, Model
import tensorflow.keras.backend as K
import json


In [18]:
# script constants
start_epoch = 30
dataset_mean_filepath = r"/content/drive/MyDrive/Colab Notebooks/pyimagesearch/bibilio2/chap11/deepergooglenet/tiny_imagenet_mean.json"
train_hdf5_path = r"/content/drive/MyDrive/ImageDataset/book2/tiny-imagenet-200/hdf5/train.hdf5"
valid_hdf5_path = r"/content/drive/MyDrive/ImageDataset/book2/tiny-imagenet-200/hdf5/val.hdf5"
checkpoint_dir = r"/content/drive/MyDrive/Colab Notebooks/pyimagesearch/bibilio2/chap11/deepergooglenet/output_artifacts/model_checkpoints"
model_filepath = r"/content/drive/MyDrive/Colab Notebooks/pyimagesearch/bibilio2/chap11/deepergooglenet/output_artifacts/model_checkpoints/model_epoch_30.h5"
#model_filepath = None

fig_output_filepath = "/content/drive/MyDrive/Colab Notebooks/pyimagesearch/bibilio2/chap11/deepergooglenet/output_artifacts/loss_curves.png"
json_output_filepath = r"/content/drive/MyDrive/Colab Notebooks/pyimagesearch/bibilio2/chap11/deepergooglenet/output_artifacts/history.json"

In [19]:
# construct the train image generator
aug = ImageDataGenerator(rotation_range=18,
                         zoom_range=0.15,
                         width_shift_range=0.2,
                         height_shift_range=0.2,
                         shear_range=0.15,
                         horizontal_flip=True,
                         fill_mode="nearest")

# load the dataset mean json
means = json.loads(open(dataset_mean_filepath, mode="r").read())
means

{'R': 122.49462207573785, 'G': 114.28757347005208, 'B': 101.35285097927517}

In [20]:
# init the preprocessors
sp = SimplePreProcessor(64, 64)
mp = MeanPreprocessor(rmean=means['R'],
                      gmean=means['G'],
                      bmean=means["B"])
iap = ImageToArrayPreprocessor()

batch_size = 64
classes = 200
# init the train and valid dataset gens
traingen = HDF5DatasetGenerator(db_path=train_hdf5_path,
                                batch_size=batch_size,
                                aug=aug,
                                preprocessors=[sp, mp, iap],
                                classes=classes)
valgen = HDF5DatasetGenerator(db_path=valid_hdf5_path,
                              batch_size=batch_size,
                              preprocessors=[sp, mp, iap],
                              classes=classes)
traingen.num_images

90000

In [22]:
# if no model checkpoint supplied then compile brand new model
model: Model
if not model_filepath:
    print(f"[INFO] compiling brand new model......")
    model = DeeperGoogleNet.build(width=64,
                                  height=64,
                                  depth=3,
                                  classes=classes,
                                  reg=0.0002)
    opt = Adam(0.001)
    model.compile(loss="categorical_crossentropy",
                  optimizer=opt,
                  metrics=['accuracy'])
    #print(model.summary())
else:
    print(f"[INFO] loading model checkpoint: {model_filepath}......")
    model = load_model(model_filepath)

    # update learning rate
    old_lr = K.get_value(model.optimizer.lr)
    new_lr = 0.00001
    K.set_value(model.optimizer.lr, new_lr)
    print(f"[INFO] old learning rate was: {old_lr}. "
          f"New learning rate set to: {K.get_value(model.optimizer.lr)}")

[INFO] loading model checkpoint: /content/drive/MyDrive/Colab Notebooks/pyimagesearch/bibilio2/chap11/deepergooglenet/output_artifacts/model_checkpoints/model_epoch_30.h5......
[INFO] old learning rate was: 9.999999747378752e-05. New learning rate set to: 9.999999747378752e-06


In [23]:
# construct 2 callbacks, one to save checkpoints every 5 epochs
# the other one to monitor training loss curves over entire process

callbacks = [
    EpochCheckpoint(outputPath=checkpoint_dir,
                    every=5,
                    startAt=start_epoch),
    TrainingMonitor(figPath=fig_output_filepath,
                    jsonPath=json_output_filepath,
                    startAt=start_epoch)]

In [ ]:
# train the network
model.fit_generator(traingen.generator(),
                    steps_per_epoch=traingen.num_images // batch_size,
                    validation_data=valgen.generator(),
                    validation_steps=valgen.num_images // batch_size,
                    epochs=30,
                    max_queue_size=10,
                    callbacks=callbacks,
                    verbose=1)
# close db
traingen.close()
valgen.close()

<ipython-input-24-009fa18d996e>:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(traingen.generator(),


Epoch 1/30
1406/1406 [==============================] - 133s 93ms/step - loss: 2.2918 - accuracy: 0.5429 - val_loss: 2.9514 - val_accuracy: 0.4216
Epoch 2/30
1406/1406 [==============================] - 131s 93ms/step - loss: 2.2808 - accuracy: 0.5447 - val_loss: 2.9457 - val_accuracy: 0.4227
Epoch 3/30
1406/1406 [==============================] - 122s 87ms/step - loss: 2.2755 - accuracy: 0.5448 - val_loss: 2.9482 - val_accuracy: 0.4226
Epoch 4/30
1406/1406 [==============================] - 123s 87ms/step - loss: 2.2681 - accuracy: 0.5466 - val_loss: 2.9460 - val_accuracy: 0.4238
Epoch 5/30
1406/1406 [==============================] - 123s 88ms/step - loss: 2.2636 - accuracy: 0.5494 - val_loss: 2.9364 - val_accuracy: 0.4264
Epoch 6/30
1406/1406 [==============================] - 123s 87ms/step - loss: 2.2550 - accuracy: 0.5488 - val_loss: 2.9452 - val_accuracy: 0.4227
Epoch 7/30
1406/1406 [==============================] - 122s 87ms/step - loss: 2.2563 - accuracy: 0.5482 - val_loss: 2